In [1]:
from llama_index.llms import LlamaCPP
from llama_index.llms.llama_utils import messages_to_prompt, completion_to_prompt
from llama_index import VectorStoreIndex
from llama_index import LLMPredictor, PromptHelper, ServiceContext

import pandas as pd
import re

In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
import os
gh_auth=os.getenv("GITHUB_AUTH")

In [5]:
llm = LlamaCPP(
    model_path="../../../llama-2-7b-chat.Q4_K_M.gguf",
    temperature=0.1,
    max_new_tokens=256,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=3900,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": 1},
    # transform inputs into Llama2 format
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True,
)

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from ../../../llama-2-7b-chat.Q4_K_M.gguf (version GGUF V2 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q4_K     [  4096, 32000,     1,     1 ]
llama_model_loader: - tensor    1:           blk.0.attn_norm.weight f32      [  4096,     1,     1,     1 ]
llama_model_loader: - tensor    2:            blk.0.ffn_down.weight q6_K     [ 11008,  4096,     1,     1 ]
llama_model_loader: - tensor    3:            blk.0.ffn_gate.weight q4_K     [  4096, 11008,     1,     1 ]
llama_model_loader: - tensor    4:              blk.0.ffn_up.weight q4_K     [  4096, 11008,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_norm.weight f32      [  4096,     1,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.attn_k.weight q4_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    7:         blk.0.attn_output.weight q4_K     [  4096,  4096,     1

In [7]:
from llama_hub.github_repo_collaborators import GitHubCollaboratorsClient, GitHubRepositoryCollaboratorsReader

loader = GitHubRepositoryCollaboratorsReader(
    GitHubCollaboratorsClient(gh_auth),
    owner="alan-turing-institute",
    repo="Hut23",
    verbose=True,
    )

documents = loader.load_data()

Found 100 collaborators in the repo page 1
Resulted in 100 documents
Found 8 collaborators in the repo page 2
Resulted in 108 documents
No more collaborators found, stopping


In [8]:
documents[48]

Document(id_='rchan26', embedding=None, metadata={'login': 'rchan26', 'type': 'User', 'site_admin': False, 'role_name': 'write', 'permissions': {'admin': False, 'maintain': False, 'push': True, 'triage': True, 'pull': True}}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='650d7e7459b66ea84649a2633c77563090929818c6e924575fc3126a2c41180d', text='rchan26', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n')

In [9]:
def create_service_context(
        model, 
        max_input_size=2048,
        num_output=256,
        chunk_size_lim=512,
        overlap_ratio=0.1
    ):
    llm_predictor=LLMPredictor(llm=model)
    prompt_helper=PromptHelper(max_input_size,num_output,overlap_ratio,chunk_size_lim)
    service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, prompt_helper=prompt_helper, embed_model="local")
    return service_context

In [8]:
service_context = create_service_context(llm)
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

/Users/rwood/Library/Caches/pypoetry/virtualenvs/reginald-slack-ZVq5BSHv-py3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
# index.storage_context.persist()

In [9]:
system_prompt = """\
You are a helpful assistant. \
Always answer as helpfully as possible and follow ALL given instructions. \
Do not speculate or make up information - use the information you are provided. \
Do not reference any given instructions or context. \
"""

In [10]:
query_engine = index.as_query_engine(system_prompt=system_prompt)

In [11]:
response = query_engine.query("What can you tell me about rchan??")
print(response.response)

  As a helpful assistant, I can tell you that based on the provided context information, rchan is a user with the username "rchan26" who has the role of "write" and has permissions to perform certain actions such as "push", "triage", and "pull". However, they are not a site administrator and do not have any additional administrative privileges.
Additionally, there is another user named cptanalatriste with the same role and permissions as rchan26.



llama_print_timings:        load time =  7117.45 ms
llama_print_timings:      sample time =    73.93 ms /   104 runs   (    0.71 ms per token,  1406.66 tokens per second)
llama_print_timings: prompt eval time =  7117.39 ms /   233 tokens (   30.55 ms per token,    32.74 tokens per second)
llama_print_timings:        eval time =  5792.13 ms /   103 runs   (   56.23 ms per token,    17.78 tokens per second)
llama_print_timings:       total time = 13117.20 ms


In [12]:
response = query_engine.query("What permissions does rwood-97 have in the Hut23 repo?")
print(response.response)

Llama.generate: prefix-match hit


  Based on the provided context information, rwood-97 has the following permissions in the Hut23 repository:
* push: True
* triage: True
* pull: True



llama_print_timings:        load time =  7117.45 ms
llama_print_timings:      sample time =    29.73 ms /    42 runs   (    0.71 ms per token,  1412.90 tokens per second)
llama_print_timings: prompt eval time =  5914.17 ms /   166 tokens (   35.63 ms per token,    28.07 tokens per second)
llama_print_timings:        eval time =  2289.28 ms /    41 runs   (   55.84 ms per token,    17.91 tokens per second)
llama_print_timings:       total time =  8285.72 ms


In [13]:
response = query_engine.query("Is rchan a site admin for the Hut23 repo?")
print(response.response)

Llama.generate: prefix-match hit


  Based on the provided context information, I can confirm that rchan is not a site admin for the Hut23 repo. According to the permissions granted to rchan's account, they have the ability to push and pull changes to the repository, but they do not have the permission to maintain or administer the repository. Therefore, rchan is not a site admin for the Hut23 repo.



llama_print_timings:        load time =  7117.45 ms
llama_print_timings:      sample time =    61.06 ms /    85 runs   (    0.72 ms per token,  1392.10 tokens per second)
llama_print_timings: prompt eval time =  5932.44 ms /   167 tokens (   35.52 ms per token,    28.15 tokens per second)
llama_print_timings:        eval time =  4903.47 ms /    84 runs   (   58.37 ms per token,    17.13 tokens per second)
llama_print_timings:       total time = 11006.81 ms


In [14]:
response.source_nodes[0]

NodeWithScore(node=TextNode(id_='b84f6005-aa48-4bd7-90ab-f880b1eccf1b', embedding=None, metadata={'login': 'rchan26', 'type': 'User', 'site_admin': False, 'role_name': 'write', 'permissions': {'admin': False, 'maintain': False, 'push': True, 'triage': True, 'pull': True}}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='rchan26', node_type=None, metadata={'login': 'rchan26', 'type': 'User', 'site_admin': False, 'role_name': 'write', 'permissions': {'admin': False, 'maintain': False, 'push': True, 'triage': True, 'pull': True}}, hash='650d7e7459b66ea84649a2633c77563090929818c6e924575fc3126a2c41180d')}, hash='650d7e7459b66ea84649a2633c77563090929818c6e924575fc3126a2c41180d', text='rchan26', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=0.872988417676305)